In [ ]:
!git clone https://github.com/dsa-playground/tvt2023.git
%cd /content/tvt2023/
!git pull
!pip install -r requirements.txt -t "tvt2023"

# Inleiding

### Use case 'Titanic'
Ruim honderd jaar geleden (1912) zonk de Titanic, vier uur nadat het schip op een ijsberg was gelopen. Slechts een derde deel van de opvarende overleefde deze ramp. Veel van deze gebeurtenis is vastgelegd, waaronder ook een dataset van passagiers. Deze dataset leent zich goed voor een introductie in de Data Science. Kan een algoritme voorspellen of een passagier overleeft? En, als je jezelf toevoegd, zou jij het dan overleefd hebben?

<!-- ![Laatste foto van de Titanic](../tvt2023/images/lastphoto_titanic.png) -->
<img src=../tvt2023/images/lastphoto_titanic.png alt="Laatste foto van de Titanic">

### Instructies omgeving
Voor deze workshop werken we in Google Colab. Dit is een online ontwikkelomgeving waarin je eenvoudig kunt experimenteren. Het notebook wat we voorbereid hebben staat al klaar. In een notebook staan cellen met ofwel code, tekst of afbeeldingen. Om een cel af te trappen (code draaien) zijn er twee mogelijkheden:
- Play-knop links van de cel
- Ctrl + Enter

### Imports & settings
Data Science heeft een sterke component met Computer Science. De meeste programmatuur zit op de achtergrond, maar om gebruik te maken van de functionaliteiten worden de functies en instellingen geladen in de cel hieronder. 

In [2]:
# Imports
import pandas as pd
from scripts.preprocess.preprocess_frontend import zie_settings, laden_data, opschonen_data, numeriek_maken_data, voeg_passagiers_toe
from scripts.EDA.eda import basis_feiten, EDA_visualisaties, correlatie_heatmap
from scripts.modeling.modeling_frontend import train_and_save_model, voorspelling_genereren

# Settings
# settings for pandas
pd.set_option("display.max.columns",None) # alle kolommen tonen
pd.set_option("display.max.rows",500)    # eerste 500 rijen tonen
pd.set_option("display.precision", 2)     # precisie van de kolommen aanpassen
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) # floats output tot 3 decimalen
pd.set_option('display.max_colwidth', None)

In [ ]:
_config = zie_settings()

Data Science is geen doel op zich. Het doel is antwoord vinden voor een vraagstuk. Om te borgen dat Data (Science) producten aansluiten bij de wensen van een klant kan men het CRISP-DM proces (Cross-Industry Standard Process for Data Mining) volgen. 


<img src=../tvt2023/images/CRISP-DM.png width=400 height=400 alt="CRISP-DM">

Dit proces doorloopt de volgende stappen:
- Business understanding: Vinden van de hypothese en context.
- Data understanding: Verzamelen van relevante data.
- Data preparation: Aanpassen data zodat deze bruikbaar is voor algoritme.
- Modeling: Opzetten/inrichten algoritme om antwoord te vinden op hypothese. 
- Evaluation: Reflecteren of resultaat model hypothese verwerpt of aanneemt.
- Deployment: Naar productieomgeving brengen (zorg dragen dat model meermaals gebruikt kan worden). 

### Business Understanding
Het vraagstuk nu concentreert zich op wel/niet overleven van de Titanic. Oftewel:
- *Kan een algoritme voorspellen of een passagier de Titanic overleefd?*

En... door onszelf toe te voegen kijken of **wij** dit hadden overleefd!

### Data Understanding
Er is een dataset beschikbaar met informatie van passagiers. Er zijn twee datasets:
1. train: Dataset met passagiers én informatie of zij wel/niet overleefd hebben
2. test: Dataset met passagiers *zonder* informatie of zij wel/niet overleefd hebben
Laten we eens kijken naar de train dataset. Draai de code door op de playknop te drukken. 

In [4]:
df_train, df_test = laden_data()
display(df_train.head())

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000,1,0,A/5 21171,7.250,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38.000,1,0,PC 17599,71.283,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.000,0,0,STON/O2. 3101282,7.925,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000,1,0,113803,53.100,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.000,0,0,373450,8.050,NaN,S


Zoals je ziet is de dataset in het Engels en soms wat cryptisch weergegeven. Om het iets eenvoudiger te maken, transformeren we de dataset naar iets begrijpelijkere taal en zetten we vergelijkbare informatie bij elkaar.

In [6]:
df_train_clean, df_test_clean = opschonen_data(df_train, df_test)
display(df_train_clean.head())

,Passagier_Id,Naam,Geslacht,Leeftijd,Opstapplaats,Aantal_kinderen,Aantal_overige_familieleden,Ticket_nummer,Ticket_klasse,Ticket_prijs,Cabine_nummer,Overleefd,Aantal_familieleden
0,1,"Braund, Mr. Owen Harris",man,22.000,Engeland,0,1,A/5 21171,Derde klas,7.250,NaN,Nee,1
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",vrouw,38.000,Frankrijk,0,1,PC 17599,Eerste klas,71.283,C85,Ja,1
2,3,"Heikkinen, Miss. Laina",vrouw,26.000,Engeland,0,0,STON/O2. 3101282,Derde klas,7.925,NaN,Ja,0
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",vrouw,35.000,Engeland,0,1,113803,Eerste klas,53.100,C123,Ja,1
4,5,"Allen, Mr. William Henry",man,35.000,Engeland,0,0,373450,Derde klas,8.050,NaN,Nee,0


De data die we nu zien heeft informatie over:
* Passagier: ID, naam, geslacht, leeftijd, opstapplaats, aantal kinderen, aantal overige familieleden, aantal familieleden
* Reisinformatie: Opstapplaats, ticket nummer, ticket klasse, cabine nummer
* Overleefd ja/nee

Wat we willen voorspellen is of mensen het overleefd hebben. De kolom 'Overleefd' is wat we noemen 'target-variabele'. De andere variabelen zijn mogelijk de verklarende variabelen. Om te kijken of er waarde zit in de variabelen, doen we een verkennende gegevensanalyse (EDA: exploratory data analysis).

In [7]:
# Moeten we nieuw berekende column niet in het midden plaatsen? 
# Zou opslagplaats ook niet bij Ticket e.d. thuis horen?

In [8]:
basis_feiten(df=df_train_clean)
EDA_visualisaties(df=df_train_clean)

Er zitten 891 passagiers in de dataset, daarvan heeft 38.38% het overleefd.
De gemiddelde leeftijd van de passagiers is 30.
De meeste passagiers zijn opgestapt in Engeland.


### Data preparation
Gezien de dataset en deze visualisaties, kunnen we de dataset nog iets meer aanpassen zodat we dit kunnen toepassen met een model. Aanpassingen welke benodigd zijn:
- Verwijderen kolommen met geen relevante data (zoals 'ticket_nummer')
- Verwijderen kolommen met veel missende data (zoals 'cabine_nummer')
- Vullen van missende waarden waar mogelijk (zoals bij 'leeftijd')
- Afronden van leeftijd (34,5 jaar = 34 jaar)
- Numeriek maken van waarden (geslacht, opstapplaats, ticket_klasse, overleefd)
- Nieuwe index maken (unieke combinatie per rij)

Dit leidt tot de volgende datasets (train & test)

In [10]:
df_train_num, df_test_num = numeriek_maken_data(df_train_clean, df_test_clean)
display(df_train_num.head(), df_test_num.head())

,,Geslacht,Leeftijd,Opstapplaats,Aantal_kinderen,Aantal_overige_familieleden,Ticket_klasse,Ticket_prijs,Overleefd,Aantal_familieleden
Passagier_Id,Naam,,,,,,,,,
1,"Braund, Mr. Owen Harris",0,22,0,0,1,3,7.250,0,1
2,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",1,38,1,0,1,1,71.283,1,1
3,"Heikkinen, Miss. Laina",1,26,0,0,0,3,7.925,1,0
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35,0,0,1,1,53.100,1,1
5,"Allen, Mr. William Henry",0,35,0,0,0,3,8.050,0,0


,,Geslacht,Leeftijd,Opstapplaats,Aantal_kinderen,Aantal_overige_familieleden,Ticket_klasse,Ticket_prijs,Aantal_familieleden
Passagier_Id,Naam,,,,,,,,
892,"Kelly, Mr. James",0,34,2,0,0,3,7.829,0
893,"Wilkes, Mrs. James (Ellen Needs)",1,47,0,0,1,3,7.000,1
894,"Myles, Mr. Thomas Francis",0,62,2,0,0,2,9.688,0
895,"Wirz, Mr. Albert",0,27,0,0,0,3,8.662,0
896,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22,0,1,1,3,12.287,2


De dataset is bijna klaar voor het toepassen van een model... we willen natuurlijk ook weten of je het zelf overleefd zou hebben. Laten we onszelf toevoegen aan de dataset.

In [ ]:
df_train_extended, df_test_extended = voeg_passagiers_toe(df_train_num, df_test_num)
display(df_train_extended.tail(), df_test_extended.tail())

### Modeling
Zoals in de presentatie besproken zijn er verschillende algoritmes om vraagstukken op te lossen. 

In [ ]:
# train_and_save_model(df=df_train_extended)

In [ ]:
df_voorspelling = voorspelling_genereren(X=df_test_extended)

In [ ]:
df_voorspelling.tail()

In [ ]:
# To do:
# - Zorgen dat bepaalde warnings e.d. niet zichtbaar zijn. 
# - Zorgen dat we weten wat bijdraagt aan de kans van overleven. 
# - Plaatjes / gifjes toevoegen
# - Tekst aanvullen
# - Evaluatie bij trainen model?

In [ ]:
# df_test_extended.tail()
# df_train_num.columns

In [ ]:
# testlist = df_train_num.columns
# m1 = ['Overleefd']
# not_in_testlist = list(set(testlist) - set(m1))
# not_in_testlist

In [ ]:
# from sklearn.model_selection import train_test_split
# X=df_train_num[['Geslacht', 'Leeftijd', 'Opstapplaats', 'Aantal_kinderen',
#        'Aantal_overige_familieleden', 'Ticket_klasse', 'Ticket_prijs']]
# y=df_train_num['Overleefd']
# # X_test = df_test_num[['Geslacht', 'Leeftijd', 'Opstapplaats', 'Aantal_kinderen',
# #        'Aantal_overige_familieleden', 'Ticket_klasse', 'Ticket_prijs']]
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#     test_size = 0.2, random_state=1)

In [ ]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.metrics import accuracy_score

# acc = []

# for i in range(1,20):
#     knn = KNeighborsClassifier(n_neighbors = i)
#     knn.fit(X_train,y_train)
#     yhat = knn.predict(X_test)
#     acc.append(accuracy_score(y_test,yhat))
#     print("For k = ",i," : ",accuracy_score(y_test,yhat))

In [ ]:
# plt.figure(figsize=(8,6))
# plt.plot(range(1,20),acc, marker = "o")
# plt.xlabel("Value of k")
# plt.ylabel("Accuracy Score")
# plt.title("Finding the right k")
# plt.xticks(range(1,20))
# plt.show()
# max(acc)

In [ ]:
# acc.index(max(acc))

In [ ]:
# import plotly.express as px

# df = df = pd.DataFrame(dict(
#     Num_neighbours = range(1,20),
#     accuracy_score = acc
# ))
# fig = px.line(df, x="Num_neighbours", y="accuracy_score")
# fig.show()

In [ ]:
# KNN = KNeighborsClassifier(n_neighbors = 3)
# KNN.fit(X,y)
# y_pred = KNN.predict(X_test)
# df_KNN = pd.DataFrame()
# # df_KNN["PassengerId"] = test2["PassengerId"]
# # df_KNN["Survived"] = y_pred

In [ ]:
# X_test['Overleefd']=y_pred

In [ ]:
# X_test

In [ ]:
# _config['preprocess']['data']['collect']['transform_multi']

toevoegen_passagiers() # pop-up venster?

EDA

Toelichting trainen / algoritme?
Iets over classificatie algoritme (x aantal opties). Nu zijn immers 2 opties: Wel / niet overleven. 

Willen we nog iets doen met train_test_split? Zo ja, dan test set hernoemen in tekst?

Trainen van model / evaluatie?

Voorspellen testset met deelnemers training + visualisaties